In [2]:
%load_ext sql
%sql sqlite:////kaggle/input/ssstore/store.csv

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Задание 1
Для каждого артиста вывести все жанры, которые есть в его песнях, и для каждого жанра вывести наиболее продолжительную песню этого артиста.

In [20]:
%%sql
SELECT ar.name, g.name, t.name, t.milliseconds FROM artists AS ar
JOIN albums al ON al.artistid=ar.artistid
JOIN tracks t ON t.albumid=al.albumid
JOIN genres g ON t.genreid=g.genreid
WHERE t.milliseconds = (
    SELECT MAX(t2.milliseconds)
    FROM tracks AS t2
    JOIN albums AS al2 ON al2.albumid = t2.albumid
    WHERE al2.artistid = ar.artistid AND t2.genreid = g.genreid
)
LIMIT 10

 * sqlite:////kaggle/input/ssstore/store.csv
   sqlite:///store.db
Done.


Name,Name_1,Name_2,Milliseconds
Accept,Rock,Princess of the Dawn,375418
AC/DC,Rock,Overdose,369319
Aerosmith,Rock,Livin' On The Edge,381231
Alanis Morissette,Rock,You Oughta Know (Alternate),491885
Alice In Chains,Rock,"Love, Hate, Love",387134
Antônio Carlos Jobim,Jazz,O Boto (Bôto),366837
Apocalyptica,Metal,Master Of Puppets,436453
Audioslave,Rock,Shadow on the Sun,343457
Audioslave,Alternative & Punk,The Curse,309786
BackBeat,Rock And Roll,Slow Down,163265


## Задание 2
Посчитать скользящее среднее выручки по годам.


In [21]:
%%sql
WITH YearRevenue AS (
    SELECT
        SUBSTR(i.InvoiceDate, 1, 4) AS Year,
        SUM(ii.Quantity * t.UnitPrice) AS Revenue
    FROM Invoices i
    JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
    JOIN tracks t ON ii.TrackId = t.TrackId
    GROUP BY Year
)
SELECT
    Year,
    Revenue,
    AVG(Revenue) OVER (ORDER BY Year ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS MovingAverage
    
FROM YearRevenue
WHERE Revenue IS NOT NULL
ORDER BY Year;

 * sqlite:////kaggle/input/ssstore/store.csv
   sqlite:///store.db
Done.


Year,Revenue,MovingAverage
2023,3205.79,3205.79
2024,1583.19,2394.49


## Задание 3

Для каждого плейлиста вывести трек с наибольшей продолжительностью, трек с наименьшей стоимостью (если у треков одинаковая стоимость, то вывести тот, который весит меньше всех), и жанр данного плейлиста (жанром плейлиста будет называться тот жанр, треков которого больше всех в плейлисте).

In [22]:
%%sql
WITH PlaylistTracks AS (
    SELECT
        p.Name,
        p.PlaylistId,
        t.Name AS TrackName,
        t.Milliseconds,
        t.UnitPrice,
        g.Name AS Genre,
        ROW_NUMBER() OVER (PARTITION BY p.PlaylistId ORDER BY t.Milliseconds DESC) AS LongestTrackRank,
        ROW_NUMBER() OVER (PARTITION BY p.PlaylistId ORDER BY t.UnitPrice, t.Milliseconds ASC) AS CheapestTrackRank,
        COUNT(t.GenreId) OVER (PARTITION BY p.PlaylistId, t.GenreId) AS GenreCount
    
    FROM playlists p
    JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId
    JOIN tracks t ON pt.TrackId = t.TrackId
    JOIN genres g ON t.GenreId = g.GenreId
)
SELECT
    pt.Name,
    pt.PlaylistId,
    MAX(CASE WHEN LongestTrackRank = 1 THEN TrackName END) AS LongestTrack,
    MAX(CASE WHEN CheapestTrackRank = 1 THEN TrackName END) AS CheapestTrack,
    
    (SELECT g.Name
     FROM genres g
     WHERE g.Name IN (
         SELECT g2.Name
         FROM PlaylistTracks pt2
         JOIN genres g2 ON pt2.Genre = g2.Name
         WHERE pt2.PlaylistId = pt.PlaylistId
         GROUP BY g2.Name
         ORDER BY COUNT(*) DESC
         LIMIT 1)
    ) AS Genre
    
FROM PlaylistTracks pt
GROUP BY pt.PlaylistId
ORDER BY pt.PlaylistId;

 * sqlite:////kaggle/input/ssstore/store.csv
   sqlite:///store.db
Done.


Name,PlaylistId,LongestTrack,CheapestTrack,Genre
From That TV Show,3,Occupation / Precipice,Your Time Is Gonna Come,TV Shows
Romantic Collection 87,4,Advance Romance,"Oh, My Love",Rock
90’s Music,5,Coma,My World,Rock
Hidden Gems,10,Occupation / Precipice,Greatest Hits,TV Shows
Brazilian Music,11,Vai Passar,A Banda,Latin
Classical,13,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo","Étude 1, In C Major - Preludio (Presto) - Liszt",Classical
The Basics Of Classic,15,"Adagio for Strings from the String Quartet, Op. 11","Aria Mit 30 Veränderungen, BWV 988 ""Goldberg Variations"": Aria",Classical
Your Sisters' Favorite Tracks,16,Alive,American Idiot,Rock
Rock Classic,17,Master Of Puppets,Run to the Hills,Metal


## Задание 4

Для каждого жанра вывести треки, которые встречались в большем числе заказов,
сумму, потраченную на данные треки, количество купленных треков и компанию, которая больше всех приобрела данную композицию.

In [23]:
%%sql
WITH TrackOrders AS (
    SELECT
        t.TrackId,
        t.Name AS TrackName,
        g.Name AS Genre,
        c.Company,
        COUNT(i.InvoiceId) AS OrderCount,
        SUM(ii.Quantity) AS TotalQuantity,
        SUM(t.UnitPrice * ii.Quantity) AS TotalSpent
    
    FROM tracks t
    JOIN genres g ON t.GenreId = g.GenreId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    JOIN customers c ON i.CustomerId = c.CustomerId
    GROUP BY t.TrackId, c.Company
),
   
RankedTracks AS (
    SELECT
        TrackName,
        Genre,
        TotalSpent,
        TotalQuantity,
        OrderCount,
        Company,
        ROW_NUMBER() OVER (PARTITION BY Genre ORDER BY OrderCount DESC) AS Rank
    
    FROM TrackOrders
)
    
SELECT 
    Genre, 
    TrackName, 
    TotalSpent, 
    TotalQuantity,
    Company
    
FROM RankedTracks
WHERE Rank = 1 AND Company IS NOT NULL
ORDER BY Genre;

 * sqlite:////kaggle/input/ssstore/store.csv
   sqlite:///store.db
Done.


Genre,TrackName,TotalSpent,TotalQuantity,Company
Alternative,Say Hello 2 Heaven,1.98,2,Oracle
Blues,First Time I Met The Blues,0.99,1,BMW
Comedy,The Convict,1.99,1,Oracle
Easy Listening,"New York, New York",2.49,1,Microsoft Corporation
Electronica/Dance,Light Years,0.99,1,JetBrains s.r.o.
Heavy Metal,Wildest Dreams,0.99,1,Google Inc.
Rock And Roll,Money,1.99,1,Google Inc.
Sci Fi & Fantasy,"Crossroads, Pt. 1",1.99,1,Intel


## Задание 5
Составить рейтинг клиентов по количеству потраченных ими денег.

In [24]:
%%sql
WITH CustomerSpending AS (
    SELECT
        c.CustomerId,
        c.FirstName || ' ' || c.LastName AS CustomerName,
        SUM(ii.Quantity * t.UnitPrice) AS TotalSpent
    
    FROM customers c
    JOIN invoices i ON c.CustomerId = i.CustomerId
    JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
    JOIN tracks t ON t.TrackId = ii.TrackId
    WHERE Company IS NOT NULL
    GROUP BY c.CustomerId
)
    
SELECT
    CustomerId,
    CustomerName,
    TotalSpent,
    RANK() OVER (ORDER BY TotalSpent DESC) AS SpendingRank
    
FROM CustomerSpending
ORDER BY SpendingRank
LIMIT 7;

 * sqlite:////kaggle/input/ssstore/store.csv
   sqlite:///store.db
Done.


CustomerId,CustomerName,TotalSpent,SpendingRank
38,Niklas Schröder,116.29,1
2,Leonie Köhler,108.34,2
23,John Gordon,105.83,3
37,Fynn Zimmermann,102.87,4
59,Puja Srivastava,100.88,5
19,Tim Goyer,97.81,6
17,Jack Smith,91.42,7
